# <font color='#4C5FDA'>**Breast Cancer Detection Based on CNNs Using Thermal Imaging** </font>

Original paper by Juan Pablo Zuluaga, Zeina Al Masry, Khaled Benaggoune, Safa Meraghni & Noureddine Zerhouni: [A CNN-based methodology for breast cancer diagnosis using thermal images](https://www.tandfonline.com/doi/full/10.1080/21681163.2020.1824685)

In [1]:
#@title **Instalar paquetes necesarios**

%%capture
! pip install torchmetrics
! pip install wandb -Uq
# ! pip install onnx

## <font color='#ECA702'>**Clonamos nuestro repo**</font>

Esto con el fin de traer todos los .py para poder entrenar 'localmente' en Colab y registrar las métricas en wandb.

In [2]:
! git clone https://github.com/gpintoruiz/Thermal-Imaging-Breast-Cancer-Detection.git

Cloning into 'Thermal-Imaging-Breast-Cancer-Detection'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 121 (delta 57), reused 70 (delta 24), pack-reused 0
Receiving objects: 100% (121/121), 2.27 MiB | 16.76 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [3]:
%cd Thermal-Imaging-Breast-Cancer-Detection/notebooks

/content/Thermal-Imaging-Breast-Cancer-Detection/notebooks


In [4]:
%ls

0.01-gpr-data-exploration-cv2.ipynb  1.00-gpr-xception-from-scratch.ipynb  utils.py
0.01-gpr-data-exploration-pil.ipynb  make_dataset.py                       validation.py
0.02-gpr-experiments-base.ipynb      test.py                               xception-one-run.yaml
0.03-gpr-initial-experiment.ipynb    train_one_run.py                      xception.py
0.04-gpr-colab-experiments.ipynb     train.py


## <font color='#ECA702'>**Configuración inicial para conectarnos con Kaggle**</font>

1. Instalamos kaggle. Para poder usar comandos de Kaggle.

In [5]:
%%capture
! pip install kaggle

Subimos nuestro token de autenticación de Kaggle (si estamos en colab, sino colocarlo en la carpeta del proyecto)

In [ ]:
from google.colab import files
files.upload()

1. Creamos los directorios de Kaggle
2. Copiamos nuestro token en .kaggle
3. Con `chmod 600` establecemos los permitos del token en 600, es decir, que solo yo tengo permisos de lectura y escritura sobre el archivo

In [7]:
! mkdir ~/kaggle

In [8]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

## <font color='#ECA702'>**Carga del dataset**</font>

Traemos el dataset [Thermal Images for Breast Cancer Diagnosis DMR-IR](https://www.kaggle.com/datasets/asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir) desde kaggle.

This dataset is a methodology for breast disease computer-aided diagnosis using dynamic thermography. The thermal images for breast tumors are classified according to DMR-IR standards.

Two types of tumors are classified in this dataset one is benign another is malignant.
- Benign: This type of tumor is usually well-defined and round or oval in shape. (non-cancerous tumor)
- Malignant: This type of tumor is usually poorly defined and irregular with lobules. (cancerous tumor)

In [9]:
%%capture
! kaggle datasets download -d asdeepak/thermal-images-for-breast-cancer-diagnosis-dmrir
! unzip thermal-images-for-breast-cancer-diagnosis-dmrir.zip

Después de descargar los datos. Debemos entender la estructura de las carpetas para poder trabajar con ellas de una mejor manera.
1. La carpeta principal `Imagens e Matrizes da Tese de Thiago Alves Elias da Silva` son todos los datos `data`.
2. La carpeta `12 Novos Casos de Testes` la podemos tomar como nuestro conjunto de prueba (`test`).
3. Mientras que la carpeta `Desenvolvimento da Metodologia` será nuestro conjunto de entrenamiento (`train`).

Luego dentro de nuestras carpetas de `train` y `test` encontramos dos categorías `DOENTES`y `SAUDAтХа├╝VEIS` o SAUDÁVEI. Los primeros son los casos malignos y los segundos benignos.

Dentro de cada una de las carpetas de pacientes saludables y enfermos se encuentran carpetas con números, cada número representa un paciente. Y para cada paciente tendremos dos carpetas más, una para las imágenes **segmentadas** en escala de grises y la otra para la matrix o mapa de calor.

Algo bueno de este dataset es que ya está dividido por pacientes, es decir, no tendremos imagenes del mismo paciente en el conjunto de entrenamiento y testeo. Por lo tanto, vamos a entrenar con N pacientes, y testear con K pacientes, que no son los mismos.

## <font color='#ECA702'>**Inicializamos el agende de wandb**</font>

### <font color='#52F17F'>**1. Nos logeamos en nuestra cuenta**</font>

In [10]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### <font color='#52F17F'>**2. Hacemos call del agente**</font>


El sweep que estoy probando acá es el [siguiente](https://github.com/gpintoruiz/Thermal-Imaging-Breast-Cancer-Detection/blob/main/notebooks/xception-one-run.yaml). Se pueden cambiar los parámetros a probar como tú quieras de acuerdo con la [documentación](https://docs.wandb.ai/guides/sweeps/define-sweep-configuration) (recomiendo solo cambiar la arquitectura para que las comparaciones entre modelos sean equivalentes).

Si no tienes ni idea qué es un sweep mira el siguiente [tutorial](https://www.youtube.com/watch?v=9zrmUIlScdY&t=1361s&ab_channel=Weights%26Biases).

El comando `--count` sirve para decirle al agente cuántos runs hacer, aplica especialmente cuando el método del sweep es `bayes` o `random`


In [11]:
! wandb agent aiuis/dip-project/epbt9jh6 --count 5

wandb: Starting wandb agent 🕵️
2024-06-09 18:36:23,481 - wandb.wandb_agent - INFO - Running runs: []
2024-06-09 18:36:23,755 - wandb.wandb_agent - INFO - Agent received command: run
2024-06-09 18:36:23,755 - wandb.wandb_agent - INFO - Agent starting run with config:
	architecture: xception
	augmented: True
	batch_size: 8
	fold: 3
	learning_rate: 0.0015528833190894193
	normalize: False
	optimizer: adam
	resize: 300
2024-06-09 18:36:23,758 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python train_one_run.py --architecture=xception --augmented=True --batch_size=8 --fold=3 --learning_rate=0.0015528833190894193 --normalize=False --optimizer=adam --resize=300
2024-06-09 18:36:28,773 - wandb.wandb_agent - INFO - Running runs: ['psadoyj7']
wandb: Currently logged in as: gpintoruiz (aiuis). Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: Tracking run with wandb version 0.17.1
wandb: Run data is saved lo